<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [50]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [51]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [52]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import spacy
nlp = spacy.load("en_core_web_lg")

In [53]:
text = yelp['text'].values.tolist()

In [54]:
def tokenize(document):
    doc = nlp(document)
    return [token.lemma.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [62]:
vect = CountVectorizer(stop_words='english')

# tokenize and build vocab
vect.fit(text) # Fit then transform. 

# transform text
dtm = vect.transform(text)

# Create a Vocabulary

In [63]:
# print(vect.get_feature_names())

In [64]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())

In [72]:
# TFIDFVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
# We want to control the value of the length of documents. 
tfidf = TfidfVectorizer(stop_words='english', min_df=0.025, max_df=.98, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Similar to fit_predict
dtm = tfidf.fit_transform(text)

# Print word counts
# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,10,15,20,30,able,absolutely,actually,ago,amazing,area,...,wonderful,work,working,worst,worth,wouldn,wrong,year,years,yelp
0,0.0,0.000000,0.239091,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.123631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.146278,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.298333,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.225699,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [74]:
# Calculate Distance of TF-IDF Vectors
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)
df = pd.DataFrame(dist_matrix)

In [75]:
df.shape

(10000, 10000)

In [76]:
df[0].sort_values(ascending=False)[:10]

0       1.000000
6899    0.512933
3714    0.501494
2131    0.456473
5465    0.439754
2865    0.433492
8470    0.408057
8531    0.404986
9036    0.403812
8411    0.401263
Name: 0, dtype: float64

In [77]:
# Nearest Neighbors
# Instantiate
from sklearn.neighbors import NearestNeighbors

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [78]:
nn.kneighbors([dtm.iloc[0].values])

(array([[0.        , 0.98698182, 0.9985046 , 1.        , 1.        ]]),
 array([[   0, 6899, 3714,  469, 3543]]))

In [79]:
random_review = [ """Chocolate was so good it gave the diabetes."""]

In [81]:
new = tfidf.transform(random_review)

In [84]:
nn.kneighbors(new.todense())

(array([[0.66237392, 0.74142786, 0.94871147, 0.99045888, 1.        ]]),
 array([[8765, 4654, 5450, 2575, 6311]]))

## Part 3: Classificatasciin
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [86]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [87]:
vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [88]:
pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [89]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

In [97]:
y = yelp['stars']
X = yelp['text']

In [114]:
# Train/Test Split
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=23)

In [115]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   30.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [116]:
pred = grid_search.predict(yelp['text'])

In [117]:
print("Total number of star predictions: ",len(pred))
print(grid_search.best_score_)
print(pred)

Total number of star predictions:  10000
0.5358208955223881
[1 5 5 ... 2 5 1]


## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [121]:
!conda install gensim -y

Solving environment: done

## Package Plan ##

  environment location: /Users/lotta/.conda/envs/NLP

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boto-2.49.0                |           py37_0         1.2 MB
    boto3-1.10.19              |             py_0          91 KB
    botocore-1.13.19           |             py_0         3.3 MB
    bz2file-0.98               |           py37_1          13 KB
    docutils-0.15.2            |           py37_0         659 KB
    gensim-3.8.0               |   py37h6440ff4_0        18.4 MB
    jmespath-0.9.4             |             py_0          22 KB
    s3transfer-0.2.1           |           py37_0          96 KB
    smart_open-1.9.0           |             py_0          59 KB
    ------------------------------------------------------------
                                           Total:        23.8 MB


In [163]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [177]:
def tokenize2(text):
    tokens = []
    for i in range(len(text)):
        tokens.append([token for token in simple_preprocess(text[i]) if token not in STOPWORDS])
    return tokens;

In [178]:
tokens = tokenize2(text)

In [188]:
def tokenize1(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [189]:
tokenize1("This is my really good review on Chocolate-Mountain Chocolate store.")

['good', 'review', 'chocolate', 'mountain', 'chocolate', 'store']

Learn the vocubalary of the yelp data:

In [182]:
id2word = Dictionary(tokens)

In [190]:
id2word.doc2bow(tokenize1("This is my really good review on Chocolate-Mountain Chocolate store."))

[(40, 1), (599, 1), (812, 1), (1540, 2), (1556, 1)]

In [196]:
len(id2word.keys())
id2word.filter_extremes(no_below=5, no_above=0.95)

In [197]:
len(id2word.keys())

7493

Create a bag of words representation of the entire corpus

In [239]:
corpus = [id2word.doc2bow(text_bow) for text_bow in tokens]

Your LDA model should be ready for estimation: 

In [200]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [201]:
lda.print_topics()

[(0,
  '0.013*"place" + 0.012*"good" + 0.009*"service" + 0.008*"time" + 0.008*"food" + 0.008*"great" + 0.006*"like" + 0.004*"love" + 0.004*"nice" + 0.004*"come"'),
 (1,
  '0.012*"place" + 0.010*"food" + 0.010*"service" + 0.009*"time" + 0.009*"great" + 0.008*"good" + 0.008*"like" + 0.005*"got" + 0.004*"ordered" + 0.004*"came"'),
 (2,
  '0.013*"food" + 0.013*"great" + 0.012*"place" + 0.009*"like" + 0.008*"service" + 0.008*"time" + 0.007*"good" + 0.006*"got" + 0.005*"ve" + 0.005*"nice"'),
 (3,
  '0.012*"good" + 0.010*"food" + 0.008*"great" + 0.007*"place" + 0.006*"like" + 0.006*"service" + 0.005*"got" + 0.005*"time" + 0.004*"went" + 0.004*"little"'),
 (4,
  '0.012*"good" + 0.011*"food" + 0.010*"place" + 0.010*"service" + 0.009*"like" + 0.009*"time" + 0.008*"great" + 0.005*"ve" + 0.004*"staff" + 0.004*"best"'),
 (5,
  '0.011*"food" + 0.010*"great" + 0.009*"place" + 0.008*"service" + 0.007*"good" + 0.006*"like" + 0.006*"time" + 0.005*"ve" + 0.004*"got" + 0.004*"order"'),
 (6,
  '0.011*"grea

In [202]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [204]:
topics = [' '.join(t[0:5]) for t in words]

In [209]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 988kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/65/e0/eb35e762802015cab1ccee04e8a277b03f1d8e53da3ec3106882ec42558b/Jinja2-2.10.3-py2.py3-none-any.whl
     |████████████████████████████████| 194kB 37.7MB/s eta 0:00:01
     |████████████████████████████████| 235kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 829kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 552kB 1.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ce/c6/f000f1af136ef74e4a95e33785921c73595c5390403f102e9b231b065b7a/MarkupSafe-1.1.1-cp37-cp37m-macosx_10_6_intel.whl
  Using cached https://files.pythonhosted.org/packages/76/bc/394ad449851729244a97857ee14d7cba61ddb268dce3db538ba2f2ba1f0f/py-1.8.0-py2.py3-none-any.whl
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=eca08e46af17717948d53b494a47afab7ed849e95c9a853c13d03961594a8925
  Stored in directory: /Users/

In [210]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [211]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/lotta/.conda/envs/NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.000274 -0.001935       1        1  10.279591
2      0.004220  0.004753       2        1  10.215107
7     -0.002189  0.003403       3        1  10.202594
6      0.000632 -0.000992       4        1  10.201081
8      0.003978 -0.001841       5        1  10.170115
9     -0.000063 -0.002497       6        1  10.087573
1      0.002093 -0.000906       7        1   9.833385
0     -0.003281 -0.002483       8        1   9.786576
3     -0.004455  0.002621       9        1   9.645413
5     -0.000660 -0.000122      10        1   9.578567, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
196   Default  4280.000000       great  4280.000000  30.0000  30.0000
39    Default  4620.000000        good  4620.000000  29.0000  29.0000
17    Default  3441.000000        like  3441.000000  28.0000  28.0000
33    Default  3418.000000        time  3418.000000  27.0000  27.0000
127   Default  4725.000000       place  4725.000000  26.0000  26.0000
109   Default  1924.000000          ve  1924.000000  25.0000  25.0000
44    Default  3631.000000     service  3631.000000  24.0000  24.0000
684   Default  1648.000000        love  1648.000000  23.0000  23.0000
72    Default  2057.000000         got  2057.000000  22.0000  22.0000
374   Default   792.000000       pizza   792.000000  21.0000  21.0000
138   Default  4924.000000        food  4924.000000  20.0000  20.0000
750   Default   700.000000       store   700.000000  19.0000  19.0000
125   Default  1433.000000      people  1433.000000  18.0000  18.0000
111   Default  1487.000000        went  1487.000000  17.0000  17.0000
361   Default  1424.000000      little  1424.000000  16.0000  16.0000
956   Default  1161.000000         new  1161.000000  15.0000  15.0000
340   Default  1235.000000   delicious  1235.000000  14.0000  14.0000
35    Default   974.000000        wait   974.000000  13.0000  13.0000
580   Default   786.000000        wasn   786.000000  12.0000  12.0000
293   Default  1453.000000         try  1453.000000  11.0000  11.0000
332   Default  1456.000000        come  1456.000000  10.0000  10.0000
45    Default  1561.000000       staff  1561.000000   9.0000   9.0000
499   Default   965.000000        told   965.000000   8.0000   8.0000
518   Default   672.000000     quality   672.000000   7.0000   7.0000
248   Default  1296.000000  experience  1296.000000   6.0000   6.0000
321   Default   683.000000     awesome   683.000000   5.0000   5.0000
38    Default  1387.000000    friendly  1387.000000   4.0000   4.0000
36    Default  1492.000000        came  1492.000000   3.0000   3.0000
708   Default   645.000000       worth   645.000000   2.0000   2.0000
679   Default   803.000000        home   803.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
846   Topic10    89.487106       clean   726.492615   0.2515  -6.1696
157   Topic10   104.359734       think   875.576599   0.2186  -6.0158
90    Topic10   113.647987       night   976.398804   0.1949  -5.9306
138   Topic10   454.035675        food  4924.865234  -0.0382  -4.5455
44    Topic10   347.982483     service  3631.111572   0.0005  -4.8115
172   Topic10   135.298096        menu  1238.375610   0.1316  -5.7562
129   Topic10    94.744736       table   807.823120   0.2025  -6.1125
45    Topic10   163.385757       staff  1561.579468   0.0883  -5.5676
379   Topic10    95.401268       sauce   819.926880   0.1945  -6.1056
127   Topic10   401.090057       place  4725.775879  -0.1210  -4.6695
679   Topic10    93.798210        home   803.953979   0.1972  -6.1225
29    Topic10   129.149078        said  1204.576782   0.1127  -5.8027
195   Topic10   123.647133       going  1148.686890   0.1167  -5.8462
72    Topic10   192.075882         got  2057.923828  -0.0259  -5.4058
38    Topic10   139

In [212]:
lda[corpus[0]]

[(0, 0.0821905),
 (1, 0.123396575),
 (2, 0.18230245),
 (3, 0.04521773),
 (4, 0.079947546),
 (5, 0.12724482),
 (6, 0.09093291),
 (7, 0.11178467),
 (8, 0.087764576),
 (9, 0.06921822)]

In [213]:
distro = [lda[d] for d in corpus]

In [215]:
def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [216]:
d_dist = {k:0 for k in range(0,15)}

In [217]:
d_dist

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0}

In [219]:
for col in yelp.columns:
    print(col)

business_id
cool
date
funny
review_id
stars
text
useful
user_id


In [230]:
# From lecture
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word, workers=4)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
from gensim.models.coherencemodel import CoherenceModel


In [236]:


def coherence(dictionary, corpus, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word, workers=4)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    
    return model_list, coherence_values


In [241]:
# Can take a long time to run.
model_list, coherence_values = coherence(dictionary=id2word, corpus=corpus, limit=40,start=2, step=6)

ValueError: One of texts or corpus has to be provided.

In [242]:
import matplotlib.pyplot as plt

limit=10; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

ValueError: x and y must have same first dimension, but have shapes (2,) and (7,)

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)